In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from implementations import *

In [36]:
def split_dsets(y, tx, pri_jet_num_idx):
    cond_null = tx[:, pri_jet_num_idx] == 0
    cond_one = tx[:, pri_jet_num_idx] == 1
    cond_plural = tx[:, pri_jet_num_idx] >= 2
    conditions = [cond_null, cond_one, cond_plural]

    dsets = [tx[cond] for cond in conditions]
    ybs = [y[cond] for cond in conditions]
    
    return dsets, ybs

In [10]:
def remove_undefined(dsets):
    clean_dsets = []
    clean_features = []

    for dset in dsets:
        # Remove constant features and features with undefined samples
        no_undefined = np.all(dset != -999, axis = 0)
        no_constant = np.any(dset != dset[0], axis = 0)
        cleaned = no_undefined * no_constant
        clean_dset = dset[:,cleaned]
        clean_dsets.append(clean_dset)
        clean_features.append(cleaned)
    
    return clean_dsets, clean_features

In [11]:
def expand_and_standardize(clean_dsets, clean_features, degree):
    standardized_dsets = []
    parameters = []

    for clean_dset in clean_dsets:
        extended_dset = build_poly(clean_dset, degree)
        standardized_dset, mean_x, std_x = extend_and_standardize(extended_dset[:,1:])
        standardized_dsets.append(standardized_dset)
        parameters.append((mean_x,std_x))
    
    return standardized_dsets, parameters

In [3]:
def compute_accuracy(y, predictions):
    N = y.size
    accuracy = 1 - (np.count_nonzero(predictions-y)/N)
    print("Accuracy: {}".format(accuracy))

In [4]:
def save_results(weights, clean_features, parameters):
    np.save('all/weights.npy', weights)
    np.save('all/clean_features.npy', clean_features)
    np.save('all/parameters.npy', parameters)

### Import Dataset

In [35]:
train_data = 'all/train.csv'
labels, input_data, ids, features = load_csv_data(train_data)
i, = np.where(features == 'PRI_jet_num')
pri_jet_num_idx = np.squeeze(i)

# Final model

In [25]:
dsets, ybs = split_dsets(labels, input_data, features)

In [26]:
clean_dsets, clean_features = remove_undefined(dsets)

In [27]:
standardized_dsets, parameters = expand_and_standardize(clean_dsets, clean_features, 8)

In [ ]:
lambdas = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    optimal_lambda = ridge_optimal_lambda(ybs[jet_num], standardized_dset)
    lambdas.append(optimal_lambda)

In [29]:
ws_RR = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    w_RR = ridge_regression(ybs[jet_num],standardized_dset,lambdas[jet_num])
    ws_RR.append(w_RR)

In [21]:
save_results(ws_RR, clean_features, parameters)

# Part 1: Without feature engineering

In [6]:
features

array(['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis',
       'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot',
       'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta',
       'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi',
       'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num',
       'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi',
       'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta',
       'PRI_jet_subleading_phi', 'PRI_jet_all_pt'], dtype='<U27')

In [155]:
training_ratio = 0.8

In [156]:
x_tr, x_te, y_tr, y_te = split_data(input_data, labels, training_ratio)

In [157]:
tx_tr, mean_tr, std_tr = extend_and_standardize(x_tr)

## Gradient descent

In [158]:
initial_w = np.zeros(x_tr.shape[1])
max_iters = 100
gamma = 0.00001

In [159]:
losses_GD, ws_GD = least_squares_GD(y_tr, x_tr, initial_w, max_iters, gamma)

/home/julien/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/julien/Documents/Epfl/Courses/ML_course/projects/project1/scripts/implementations.py:44: RuntimeWarning: overflow encountered in square
  return 1/2*np.mean(e**2)


In [53]:
predictions_GD = predict_labels(w_GD,x_te)
compute_accuracy(y_te,predictions_GD)

/home/julien/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/home/julien/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less_equal
  after removing the cwd from sys.path.


## Stochastic gradient descent

## Least squares

### No standardization

In [160]:
loss, w_LS = least_squares(y_tr,x_tr)
predictions = predict_labels(w_LS,x_te)

In [161]:
compute_accuracy(y_te,predictions)

Accuracy: 0.74402


### With standardization

In [28]:
tx_te, _, _ = extend_and_standardize(x_te, mean_tr, std_tr)

In [29]:
loss, w_LS = least_squares(y_tr,tx_tr)
predictions = predict_labels(w_LS,tx_te)

In [30]:
compute_accuracy(y_te,predictions)

Accuracy: 0.74468


## Ridge regression

### Use cross-validation to find good hyperparameter

In [31]:
seed = 1
k_fold = 4
k_indices = build_k_indices(y_tr, k_fold, seed)
lambda_ = 0.001

In [45]:
optimal_lambda = ridge_optimal_lambda(y_tr,x_tr)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29


In [50]:
w_rr = ridge_regression(y_tr,x_tr,optimal_lambda)

In [36]:
optimal_lambda

0.006210169418915616

In [51]:
optimal_lambda

0.006210169418915616

In [52]:
predictions = predict_labels(w_rr,x_te)

In [38]:
compute_accuracy(y_te,predictions)

Accuracy: 0.744


In [53]:
compute_accuracy(y_te,predictions)

Accuracy: 0.744


## Logistic regression

In [164]:
initial_w = np.zeros(x_tr.shape[1])
max_iters = 1000
gamma = 0.7
y_tr_log = np.ones(y_tr.size)
y_tr_log[y_tr == -1] = 0

In [165]:
loss, w_logistic = logistic_regression(y_tr_log, x_tr, initial_w, max_iters, gamma)

/home/julien/Documents/Epfl/Courses/ML_course/projects/project1/scripts/implementations.py:55: RuntimeWarning: divide by zero encountered in log
  loss = (-y * np.log(pred) - (1. - y) * np.log(1. - pred)).mean()
/home/julien/Documents/Epfl/Courses/ML_course/projects/project1/scripts/implementations.py:55: RuntimeWarning: invalid value encountered in multiply
  loss = (-y * np.log(pred) - (1. - y) * np.log(1. - pred)).mean()


Current iteration=0, loss=nan
Current iteration=100, loss=nan
Current iteration=200, loss=nan
Current iteration=300, loss=nan
Current iteration=400, loss=nan
Current iteration=500, loss=nan
Current iteration=600, loss=nan
Current iteration=700, loss=nan
Current iteration=800, loss=nan
Current iteration=900, loss=nan


In [166]:
predictions = predict_labels(w_logistic, x_te)
compute_accuracy(y_te,predictions)

Accuracy: 0.67828


## Regularized logistic regression

In [ ]:
initial_w = np.zeros(x_tr.shape[1])
max_iters = 100
gamma = 0.7 # 0.01
y_logistic = np.ones(y_tr.size)
y_logistic[y_tr == -1] = 0
optimal_lambda = logistic_optimal_lambda(y_logistic, x_tr, initial_w, max_iters, gamma)

In [156]:
loss, w_reg_logistic = reg_logistic_regression(y_tr_log, x_tr, optimal_lambda, initial_w, max_iters, gamma)

/home/julien/Documents/Epfl/Courses/ML_course/projects/project1/scripts/implementations.py:116: RuntimeWarning: divide by zero encountered in log
  loss = (-y * np.log(pred) - (1 - y) * np.log(1 - pred)).mean()
/home/julien/Documents/Epfl/Courses/ML_course/projects/project1/scripts/implementations.py:116: RuntimeWarning: invalid value encountered in multiply
  loss = (-y * np.log(pred) - (1 - y) * np.log(1 - pred)).mean()


Current iteration=0, loss=nan
Current iteration=100, loss=nan
Current iteration=200, loss=nan
Current iteration=300, loss=nan
Current iteration=400, loss=nan
Current iteration=500, loss=nan
Current iteration=600, loss=nan
Current iteration=700, loss=nan
Current iteration=800, loss=nan
Current iteration=900, loss=nan


In [158]:
predictions = predict_labels(w_reg_logistic, x_te)
compute_accuracy(y_te,predictions)

Accuracy: 0.7079599999999999


# Part 2: EDA and feature engineering

### Split data for testing

In [6]:
training_ratio = 0.8
x_tr, x_te, y_tr, y_te = split_data(input_data, labels, training_ratio)

In [ ]:
dsets, ybs = split_dsets(y_tr, x_tr, features)

In [ ]:
clean_dsets, clean_features = remove_undefined(dsets)

In [ ]:
standardized_dsets, parameters = expand_and_standardize(clean_dsets, clean_features, 2)

## Gradient descent

In [60]:
max_iters = 1000
gamma = 0.1

In [64]:
ws_GD = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    initial_w = np.zeros(standardized_dset.shape[1])
    losses_GD, w_GD = least_squares_GD(ybs[jet_num], standardized_dset, initial_w, max_iters, gamma)
    ws_GD.append(w_GD)

In [65]:
predictions = model_predictions(x_te, ws_GD, pri_jet_num_idx, clean_features, parameters, 8)

In [63]:
compute_accuracy(y_te,predictions)

Accuracy: 0.78418


In [66]:
compute_accuracy(y_te,predictions)

Accuracy: 0.78418


Second score, after replacing -999 in DER_mass_MMC by the defined mean.

In [44]:
compute_accuracy(y_te,predictions)

Accuracy: 0.72386


Conclusion: bit worse

Third score, after replacing -999 in DER_mass_MMC by the defined median.

In [70]:
compute_accuracy(y_te,predictions)

Accuracy: 0.72848


## Stochastic Gradient descent

In [81]:
max_iters = 1000
gamma = 0.1
batch_size = 100

In [82]:
ws_SGD = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    initial_w = np.zeros(standardized_dset.shape[1])
    loss_SGD, w_SGD = least_squares_SGD(ybs[jet_num], standardized_dset, initial_w, batch_size, max_iters, gamma)
    ws_SGD.append(w_SGD)

In [83]:
predictions = model_predictions(x_te, ws_SGD, pri_jet_num_idx, clean_features, parameters, 8)

In [76]:
compute_accuracy(y_te,predictions)

Accuracy: 0.70154


In [84]:
compute_accuracy(y_te,predictions)

Accuracy: 0.7794


## Least squares

In [174]:
ws_LS = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    loss, w = least_squares(ybs[jet_num],standardized_dset)
    ws_LS.append(w)

In [175]:
predictions = model_predictions(x_te, ws_LS, pri_jet_num_idx, clean_features, parameters, 8)

In [19]:
compute_accuracy(y_te,predictions)

Accuracy: 0.78556


Second score, after replacing -999 in DER_mass_MMC by the defined mean.

In [69]:
compute_accuracy(y_te,predictions)

Accuracy: 0.75986


Third score, after replacing -999 in DER_mass_MMC by the defined median.

In [50]:
compute_accuracy(y_te,predictions)

Accuracy: 0.7598


Handle outliers

In [176]:
compute_accuracy(y_te,predictions)

Accuracy: 0.79222


In [90]:
ws_LS = []
for jet_num, extended_dset in enumerate(extended_dsets):
    loss, w = least_squares(ybs[jet_num],extended_dset)
    ws_LS.append(w)

In [91]:
predictions = model_predictions(x_te, ws_LS, pri_jet_num_idx, clean_features, parameters, 8)
compute_accuracy(y_te, predictions)

Accuracy: 0.76336


## Ridge regression

In [191]:
lambdas = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    optimal_lambda = ridge_optimal_lambda(ybs[jet_num], standardized_dset)
    lambdas.append(optimal_lambda)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Ite

In [35]:
lambdas

[1.0, 1.0, 0.05736152510448681]

Degree 3 polynomial 

In [192]:
lambdas

[0.14873521072935117, 1.0, 0.002395026619987486]

In [193]:
ws_RR = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    w_RR = ridge_regression(ybs[jet_num],standardized_dset,lambdas[jet_num])
    ws_RR.append(w_RR)

In [194]:
predictions = model_predictions(x_te, ws_RR, pri_jet_num_idx, clean_features, parameters, 8)

In [38]:
compute_accuracy(y_te,predictions)

Accuracy: 0.7854


In [58]:
save_results(ws_RR, clean_features, parameters)

Degree 3 polynomial

In [195]:
compute_accuracy(y_te,predictions)

Accuracy: 0.79234


## Logistic regression

In [24]:
ws_LR = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    initial_w = np.zeros(standardized_dset.shape[1])
    max_iters = 2000
    gamma = 0.7 # 0.01
    y_logistic = np.ones(ybs[jet_num].size)
    y_logistic[ybs[jet_num] == -1] = 0
    loss, w_LR = logistic_regression(y_logistic, standardized_dset, initial_w, max_iters, gamma)
    ws_LR.append(w_LR)

Current iteration=0, loss=0.5853413172803463
Current iteration=100, loss=0.38242539449182134
Current iteration=200, loss=0.3758365528197574
Current iteration=300, loss=0.37406757141223684
Current iteration=400, loss=0.3733865214630594
Current iteration=500, loss=0.37304664080949995
Current iteration=600, loss=0.3728418967625082
Current iteration=700, loss=0.37270160860474555
Current iteration=800, loss=0.3725955789555245
Current iteration=900, loss=0.3725094992688114
Current iteration=1000, loss=0.37243618360400654
Current iteration=1100, loss=0.3723717370451153
Current iteration=1200, loss=0.37231385658826005
Current iteration=1300, loss=0.37226106593147135
Current iteration=1400, loss=0.37221235451715207
Current iteration=1500, loss=0.3721669954163542
Current iteration=1600, loss=0.37212444615370477
Current iteration=1700, loss=0.3720842905543233
Current iteration=1800, loss=0.37204620238916103
Current iteration=1900, loss=0.3720099214328091
Current iteration=0, loss=0.62305297445425

In [25]:
predictions = model_predictions(x_te, ws_LR, pri_jet_num_idx, clean_features, parameters, 3)

In [26]:
compute_accuracy(y_te,predictions)

Accuracy: 0.8017


In [28]:
save_results(ws_LR, clean_features, parameters)

In [99]:
compute_accuracy(y_te,predictions)

Accuracy: 0.8017


In [204]:
save_results(ws_LR, clean_features, parameters)

Second score, after replacing -999 in DER_mass_MMC by the defined mean.

In [235]:
compute_accuracy(y_te,predictions)

Accuracy: 0.76258


Third score, after replacing -999 in DER_mass_MMC by the defined median.

In [225]:
compute_accuracy(y_te,predictions)

Accuracy: 0.76196


Logitic regression, degree 3

In [179]:
compute_accuracy(y_te,predictions)

Accuracy: 0.80266


## Regularized logistic regression

In [180]:
lambdas = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    initial_w = np.zeros(standardized_dset.shape[1])
    max_iters = 2000
    gamma = 0.7 # 0.01
    y_logistic = np.ones(ybs[jet_num].size)
    y_logistic[ybs[jet_num] == -1] = 0
    optimal_lambda = logistic_optimal_lambda(y_logistic, standardized_dset, initial_w, max_iters, gamma)
    lambdas.append(optimal_lambda)

Iteration 0
Current iteration=0, loss=0.5840898688325303
Current iteration=100, loss=0.38302967217440864
Current iteration=200, loss=0.3778226394034337
Current iteration=300, loss=0.3761220817626732
Current iteration=400, loss=0.3753605022566267
Current iteration=500, loss=0.3749101570804421
Current iteration=600, loss=0.3745743367064042
Current iteration=700, loss=0.3742894528664899
Current iteration=800, loss=0.3740320084918895
Current iteration=900, loss=0.3737923776663226
Current iteration=1000, loss=0.37356622529283084
Current iteration=1100, loss=0.3733514161762931
Current iteration=1200, loss=0.3731467994444135
Current iteration=1300, loss=0.37295168461626355
Current iteration=1400, loss=0.3727655926181169
Current iteration=1500, loss=0.3725954781543667
Current iteration=1600, loss=0.37248753313593114
Current iteration=1700, loss=0.37240281837422784
Current iteration=1800, loss=0.372333537129307
Current iteration=1900, loss=0.372275597533538
Current iteration=0, loss=0.372819366

Current iteration=1200, loss=0.3737612025388063
Current iteration=1300, loss=0.3737525987889186
Current iteration=1400, loss=0.373745115368474
Current iteration=1500, loss=0.37373857317043224
Current iteration=1600, loss=0.3737328265017454
Current iteration=1700, loss=0.37372775613755144
Current iteration=1800, loss=0.3737232639365049
Current iteration=1900, loss=0.3737192686414367
Current iteration=0, loss=0.3744035082446261
Current iteration=100, loss=0.3742425211474675
Current iteration=200, loss=0.37421282751410107
Current iteration=300, loss=0.3741943686026007
Current iteration=400, loss=0.3741809124827378
Current iteration=500, loss=0.3741702374381993
Current iteration=600, loss=0.3741613693472125
Current iteration=700, loss=0.3741538028137963
Current iteration=800, loss=0.37414723721688853
Current iteration=900, loss=0.3741414738002689
Current iteration=1000, loss=0.37413637054914356
Current iteration=1100, loss=0.37413182032058
Current iteration=1200, loss=0.37412773924543175
C

Current iteration=400, loss=0.37709997602409634
Current iteration=500, loss=0.3770930579093503
Current iteration=600, loss=0.37708764266208633
Current iteration=700, loss=0.3770832704055291
Current iteration=800, loss=0.377079668589509
Current iteration=900, loss=0.37707665923404166
Current iteration=1000, loss=0.3770741179629736
Current iteration=1100, loss=0.3770719536824448
Current iteration=1200, loss=0.37707009745311476
Current iteration=1300, loss=0.37706849586322516
Current iteration=1400, loss=0.37706710679869815
Current iteration=1500, loss=0.37706589659109285
Current iteration=0, loss=0.3768460897038585
Current iteration=100, loss=0.37659334391345856
Current iteration=200, loss=0.37649548845717246
Current iteration=300, loss=0.37643051013715867
Current iteration=400, loss=0.3763819293255297
Current iteration=500, loss=0.37634314152550646
Current iteration=600, loss=0.37631094032118345
Current iteration=700, loss=0.3762835629998467
Current iteration=800, loss=0.376260025979830

Current iteration=200, loss=0.384411037396742
Current iteration=300, loss=0.3843672710134138
Current iteration=400, loss=0.3843448012126863
Current iteration=500, loss=0.3843318709200725
Current iteration=600, loss=0.3843238487344807
Current iteration=700, loss=0.38431860919421956
Current iteration=800, loss=0.38431506155883605
Current iteration=900, loss=0.38431259792544803
Current iteration=1000, loss=0.3843108565479117
Current iteration=1100, loss=0.3843096104994086
Current iteration=0, loss=0.3846322630824754
Current iteration=100, loss=0.3845001775624101
Current iteration=200, loss=0.3844870473640092
Current iteration=300, loss=0.38448090102064086
Current iteration=400, loss=0.3844772220071955
Current iteration=500, loss=0.38447476574782774
Current iteration=600, loss=0.3844730465623656
Current iteration=700, loss=0.3844718145216981
Current iteration=0, loss=0.38417465913602916
Current iteration=100, loss=0.3839795039284595
Current iteration=200, loss=0.3839412769717242
Current it

Current iteration=0, loss=0.43729221371055027
Current iteration=100, loss=0.43718622116887235
Current iteration=0, loss=0.4377789241066053
Current iteration=100, loss=0.4376425794781645
Iteration 16
Current iteration=0, loss=0.45182851626211523
Current iteration=100, loss=0.45047551113571044
Current iteration=0, loss=0.4507889873984481
Current iteration=0, loss=0.44942321428902116
Current iteration=0, loss=0.45000485846571175
Current iteration=100, loss=0.4498912483869394
Iteration 17
Current iteration=0, loss=0.46562193457537915
Current iteration=0, loss=0.4644643518310449
Current iteration=0, loss=0.4630423403995487
Current iteration=0, loss=0.46369290368888705
Iteration 18
Current iteration=0, loss=0.4808717385214998
Current iteration=0, loss=0.47956451503427044
Current iteration=0, loss=0.47811620202531985
Current iteration=0, loss=0.4788072251794409
Iteration 19
Current iteration=0, loss=0.49740092116064577
Current iteration=0, loss=0.49596240596993596
Current iteration=0, loss=0.

...

Current iteration=200, loss=0.4981435360920889
Current iteration=300, loss=0.4981327574591921
Current iteration=400, loss=0.4981283697319265
Current iteration=500, loss=0.4981264947769446
Current iteration=0, loss=0.4961345593295948
Current iteration=100, loss=0.4956751971734358
Current iteration=200, loss=0.4956503191482764
Current iteration=300, loss=0.4956435524391135
Current iteration=400, loss=0.4956410356634344
Iteration 11
Current iteration=0, loss=0.5037772053776347
Current iteration=100, loss=0.5025703236132024
Current iteration=200, loss=0.5024141414757165
Current iteration=300, loss=0.5023811846429596
Current iteration=400, loss=0.5023737127598864
Current iteration=0, loss=0.49897537876223125
Current iteration=100, loss=0.49870686917317336
Current iteration=200, loss=0.4986942541978005
Current iteration=300, loss=0.4986909147270251
Current iteration=0, loss=0.5035925833496077
Current iteration=100, loss=0.5032126675875356
Current iteration=200, loss=0.5031911760544227
Curren

Current iteration=1200, loss=0.5924910677398454
Current iteration=1300, loss=0.5924910677398454
Current iteration=1400, loss=0.5924910677398454
Current iteration=1500, loss=0.5924910677398454
Current iteration=1600, loss=0.5924910677398454
Current iteration=1700, loss=0.5924910677398454
Current iteration=1800, loss=0.5924910677398454
Current iteration=1900, loss=0.5924910677398454
Current iteration=0, loss=0.5897943800827146
Current iteration=100, loss=0.5882203460925397
Current iteration=200, loss=0.588220346092539
Current iteration=300, loss=0.588220346092539
Current iteration=400, loss=0.588220346092539
Current iteration=500, loss=0.588220346092539
Current iteration=600, loss=0.588220346092539
Current iteration=700, loss=0.5882203460925391
Current iteration=800, loss=0.588220346092539
Current iteration=900, loss=0.5882203460925391
Current iteration=1000, loss=0.5882203460925391
Current iteration=1100, loss=0.5882203460925391
Current iteration=1200, loss=0.588220346092539
Current ite

Current iteration=500, loss=0.6570901402296804
Current iteration=600, loss=0.6570901402296804
Current iteration=700, loss=0.6570901402296804
Current iteration=800, loss=0.6570901402296804
Current iteration=900, loss=0.6570901402296804
Current iteration=1000, loss=0.6570901402296804
Current iteration=1100, loss=0.6570901402296804
Current iteration=1200, loss=0.6570901402296804
Current iteration=1300, loss=0.6570901402296804
Current iteration=1400, loss=0.6570901402296804
Current iteration=1500, loss=0.6570901402296804
Current iteration=1600, loss=0.6570901402296804
Current iteration=1700, loss=0.6570901402296804
Current iteration=1800, loss=0.6570901402296804
Current iteration=1900, loss=0.6570901402296804
Current iteration=0, loss=0.6593558066188273
Current iteration=100, loss=0.6621331697617637
Current iteration=200, loss=0.6621331697617637
Current iteration=300, loss=0.6621331697617637
Current iteration=400, loss=0.6621331697617637
Current iteration=500, loss=0.6621331697617637
Curre

Current iteration=1800, loss=0.8998801566885586
Current iteration=1900, loss=0.8998801566885586
Current iteration=0, loss=0.9015598520968873
Current iteration=100, loss=0.9062051010918881
Current iteration=200, loss=0.9062051010918881
Current iteration=300, loss=0.9062051010918881
Current iteration=400, loss=0.9062051010918881
Current iteration=500, loss=0.9062051010918881
Current iteration=600, loss=0.9062051010918881
Current iteration=700, loss=0.9062051010918881
Current iteration=800, loss=0.9062051010918881
Current iteration=900, loss=0.9062051010918881
Current iteration=1000, loss=0.9062051010918881
Current iteration=1100, loss=0.9062051010918881
Current iteration=1200, loss=0.9062051010918881
Current iteration=1300, loss=0.9062051010918881
Current iteration=1400, loss=0.9062051010918881
Current iteration=1500, loss=0.9062051010918881
Current iteration=1600, loss=0.9062051010918881
Current iteration=1700, loss=0.9062051010918881
Current iteration=1800, loss=0.9062051010918881
Curr

Current iteration=1200, loss=3.644812823378978
Current iteration=1300, loss=3.644812823378978
Current iteration=1400, loss=3.644812823378978
Current iteration=1500, loss=3.644812823378978
Current iteration=1600, loss=3.644812823378978
Current iteration=1700, loss=3.644812823378978
Current iteration=1800, loss=3.644812823378978
Current iteration=1900, loss=3.644812823378978
Current iteration=0, loss=3.6074820923598656
Current iteration=100, loss=3.5701763569148643
Current iteration=200, loss=3.5701763568925013
Current iteration=300, loss=3.5701763568925013
Current iteration=400, loss=3.5701763568925013
Current iteration=500, loss=3.5701763568925013
Current iteration=600, loss=3.5701763568925013
Current iteration=700, loss=3.5701763568925013
Current iteration=800, loss=3.5701763568925013
Current iteration=900, loss=3.5701763568925013
Current iteration=1000, loss=3.5701763568925013
Current iteration=1100, loss=3.5701763568925013
Current iteration=1200, loss=3.5701763568925013
Current iter

In [145]:
lambdas

[0.00013738237958832623, 0.0001, 0.0001]

Degree 3 polynomial expansion

In [181]:
lambdas

[0.00013738237958832623, 0.00013738237958832623, 0.00013738237958832623]

In [263]:
lambdas

[0.0001, 0.0001, 0.0001]

In [182]:
ws_RLR = []
for jet_num, standardized_dset in enumerate(standardized_dsets):
    initial_w = np.zeros(standardized_dset.shape[1])
    max_iters = 2000
    gamma = 0.7 # 0.01
    y_logistic = np.ones(ybs[jet_num].size)
    y_logistic[ybs[jet_num] == -1] = 0
    loss, w_RLR = reg_logistic_regression(y_logistic, standardized_dset, lambdas[jet_num], initial_w, max_iters, gamma)
    ws_RLR.append(w_RLR)

Current iteration=0, loss=0.5836319739997964
Current iteration=100, loss=0.38299140175875035
Current iteration=200, loss=0.37803365822497376
Current iteration=300, loss=0.37645182879044486
Current iteration=400, loss=0.3757436234372618
Current iteration=500, loss=0.3753383539575887
Current iteration=600, loss=0.37504392132409753
Current iteration=700, loss=0.3747993795321187
Current iteration=800, loss=0.3745825514463692
Current iteration=900, loss=0.3743843054702021
Current iteration=1000, loss=0.3742003263408803
Current iteration=1100, loss=0.3740282770401534
Current iteration=1200, loss=0.37386669992413457
Current iteration=1300, loss=0.37371456864327324
Current iteration=1400, loss=0.37357109541035727
Current iteration=1500, loss=0.3734356423191489
Current iteration=1600, loss=0.3733076760382906
Current iteration=1700, loss=0.37320222103428036
Current iteration=1800, loss=0.3731331670545731
Current iteration=1900, loss=0.3730784831660644
Current iteration=0, loss=0.6214390936229288

In [183]:
predictions = model_predictions(x_te, ws_RLR, pri_jet_num_idx, clean_features, parameters, 3)

In [153]:
compute_accuracy(y_te,predictions)

Accuracy: 0.80052


Degree 3 polynomial expansion

In [184]:
compute_accuracy(y_te,predictions)

Accuracy: 0.8017799999999999
